Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).  

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
2. Обучите по коллекции предложений word2vec

3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    тест на определение ближайших слов
    тест на аналогии (мужчина – король : женщина – королева)
    тест на определение лишнего слова.

3. Постройте несколько визуализаций:
    TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах. Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

---

In [1]:
import re
import json
import bz2
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
import gensim
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
from gensim.models import word2vec
import matplotlib.pyplot as plt
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:14, 1032.20it/s]


In [3]:
responses[0]

{'author': 'uhnov1',
 'bank_license': 'лицензия № 2562',
 'bank_name': 'Бинбанк',
 'city': 'г. Москва',
 'datetime': '2015-06-08 12:50:54',
 'num_comments': 0,
 'rating_grade': None,
 'rating_not_checked': False,
 'text': 'Добрый день! Я не являюсь клиентом банка и поручителем по кредитному договору, а также не являюсь каким-либо другим лицом, письменно  оформившим отношения с банком по поводу урегулирования чьей-либо  задолженности.  Начиная с марта 2015 года начали приходить бесконечные письма из ООО "Примо коллект"на мой адрес: город Москва, Уваровский переулок, дом 10, квартира 111, с угрозами о возбуждении уголовного дела в отношении гражданина Филиппова Эдуарда Владимировича, который уклоняется от уплаты взятых им кредитов: договор № 81014 от 20.10.2013 года и договор № 2464946 от 09.10.2014 года. Со всей ответственностью\xa0 хочу Вас заверить, что вышеуказанный гражданин, которого Вы разыскиваете, мне не знаком и никогда в моем адресе не был зарегистрирован. Каким образом Вы не 

In [4]:
df = pd.DataFrame(responses)
df.head()

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.

In [5]:
regex = re.compile("[А-Яа-я]+")

In [6]:
my_stopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', 'который', 'мой', 'ваш', 'год', 'г', 'р', 'руб', 'рубль']
my_stopwords.remove('хорошо')

In [7]:
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [9]:
def remove_stopwords(text, mystopwords=my_stopwords):
    try:
        return ' '.join([token for token in text.split() if token not in mystopwords])
    except:
        return ''

In [8]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
analyzer = MorphAnalyzer()

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 8.2MB 6.0MB/s 


In [10]:
def lemmatize(text, analyzer=analyzer):
    try:
        return ' '.join([analyzer.parse(word)[0].normal_form for word in text.split()])
    except:
        return ' '

In [11]:
%%time 
df.text = df.text.str.lower()
df.text = df.text.apply(words_only)
df.text = df.text.apply(remove_stopwords)
df.text = df.text.apply(lemmatize)
df.text = df.text.apply(remove_stopwords)
sentences = []  
for text in df.text.values:
  sentences.append(text.split())

CPU times: user 1h 8min 6s, sys: 6.15 s, total: 1h 8min 12s
Wall time: 1h 8min 20s


In [12]:
len(sentences)

153499

2. Обучите по коллекции предложений word2vec.

In [13]:
%time w2v_model = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

CPU times: user 10min 11s, sys: 1.79 s, total: 10min 13s
Wall time: 5min 18s


In [14]:
len(w2v_model.wv.vocab)

24374

3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    - тест на определение ближайших слов
    - тест на аналогии (мужчина – король : женщина – королева)
    - тест на определение лишнего слова.

Удачные примеры ближайших слов:

In [15]:
print(w2v_model.wv.most_similar("сбербанк", topn=5))

[('сбер', 0.5491206049919128), ('сб', 0.527484118938446), ('осб', 0.5037587881088257), ('сберкнижка', 0.45274439454078674), ('сбербанковский', 0.4447806775569916)]


In [16]:
print(w2v_model.wv.most_similar("девушка", topn=5))

[('барышня', 0.7482657432556152), ('сотрудница', 0.7084072232246399), ('девочка', 0.6972975134849548), ('юноша', 0.6539922952651978), ('парень', 0.6343950033187866)]


In [17]:
print(w2v_model.wv.most_similar("оценка", topn=5))

[('единица', 0.6916276216506958), ('троечка', 0.6713801622390747), ('двойка', 0.6604766249656677), ('четвёрка', 0.6530194282531738), ('пятёрка', 0.6436413526535034)]


Неудачные примеры ближайших слов:

In [18]:
print(w2v_model.wv.most_similar("продукт", topn=5))

[('преимущество', 0.5517234802246094), ('активно', 0.4956623315811157), ('плюшка', 0.4708665609359741), ('линейка', 0.4629264771938324), ('инструмент', 0.44708263874053955)]


In [19]:
print(w2v_model.wv.most_similar("сдаться", topn=5))

[('шутить', 0.5694933533668518), ('сдержаться', 0.5146478414535522), ('вытерпеть', 0.5089338421821594), ('перебить', 0.508220911026001), ('сжалиться', 0.5078040957450867)]


Удачные примеры аналогий:

In [20]:
print(w2v_model.wv.most_similar(positive=["девушка", "сотрудник"], negative=["парень"], topn=1))

[('сотрудница', 0.6921380758285522)]


In [21]:
print(w2v_model.wv.most_similar(positive=["карта", "банкомат"], negative=["деньга"], topn=1))

[('боп', 0.4625760018825531)]


In [22]:
print(w2v_model.wv.most_similar(positive=["сотрудник", "клиент"], negative=["вкладчик"], topn=1))

[('специалист', 0.5243853330612183)]


Неудачные примеры аналогий:

In [23]:
print(w2v_model.wv.most_similar(positive=["сотрудник", "работать"], negative=["клиент"], topn=1))

[('сотрудница', 0.4596219062805176)]


In [24]:
print(w2v_model.wv.most_similar(positive=["банк", "клиент"], negative=["деньга"], topn=1))

[('проявлять', 0.42148151993751526)]


In [25]:
print(w2v_model.wv.most_similar(positive=["территория", "страна"], negative=["офис"], topn=1))

[('снг', 0.6132520437240601)]


3. Постройте несколько визуализаций:
    TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    задайте координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  
    Более формально: берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.

In [26]:
top_words = []

fd = nltk.FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(500):
    top_words.append(w[0])

100%|██████████| 153499/153499 [00:17<00:00, 8775.79it/s] 


In [27]:
top_words_vec = w2v_model[top_words]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [29]:
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [30]:
!pip install bokeh

In [31]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

# Отобразим 500 самых частых слов и приблизим к какому-нибудь кластеру
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (v2w model, top500 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Можно приблизить к кластеру с названиями месяцев.

Возьмём названия нескольких банков и из ближайших 10 слов к этим названиям добавим названия других банков.

In [107]:
banks_init = ['втб', 'газпромбанк', 'уралсиб', 'райффайзенбанк', 'возрождение', 'транскредитбанк', 'промсвязьбанк', 'росбанк', 'мкб', 'юникредит']
banks = []
for bank in banks_init:
  for i in range(10):
    sim_bank = w2v_model.wv.most_similar(bank, topn=10)[i][0]
    if sim_bank not in banks:
      banks.append(sim_bank)
banks.append('сбербанк')
banks.remove('аккредитовать')

In [108]:
banks_vec = w2v_model[banks]
banks_vec_new = np.zeros((len(banks), 2))

# Зададим векторы для нового пространства
x_ = w2v_model['хорошо'] - w2v_model['плохо']
y_ = w2v_model['быстро'] - w2v_model['медленно']

# Получим координаты банков относительно векторов нового пространства
for i in range(len(banks)):
  banks_vec_new[i][0] = np.dot(banks_vec[i], x_)
  banks_vec_new[i][1] = np.dot(banks_vec[i], y_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [111]:
# Отобразим банки на новой плоскости
s = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec x_axe: 'bad' - 'good', y_axe: 'slow' - 'fast' (v2w model, banks)")



source = ColumnDataSource(data=dict(x1=banks_vec_new[:,0],
                                    x2=banks_vec_new[:,1],
                                    names=banks))

s.scatter(x="x1", y="x2", size=8, source=source)

s.plot_height = 800
s.plot_width = 1000

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
s.add_layout(labels)

show(s)